In [ ]:
## LIBRARIES ##
from gemseo import (
    configure_logger,
    create_design_space,
    create_scenario,
    create_surrogate,
)
from gemseo.algos.parameter_space import ParameterSpace
from gemseo.mlearning.quality_measures.r2_measure import R2Measure
from gemseo.mlearning.quality_measures.rmse_measure import RMSEMeasure
from gemseo.uncertainty.sensitivity.sobol.analysis import SobolAnalysis
from gemseo_mlearning.api import sample_discipline

from lh2pac.gemseo.discipline import H2TurboFan
from lh2pac.gemseo.utils import draw_aircraft, get_aircraft_data
from lh2pac.marilib.utils import unit


import matplotlib.pyplot as plt
import pprint
from gemseo.uncertainty.sensitivity.morris.analysis import MorrisAnalysis

# PART 3 Surrogate modeling and robust optimization
from gemseo_umdo.scenarios.umdo_scenario import UMDOScenario

# 1. Comparaison entre le modèle H2TURBOFAN et un modèle de substitution

## 1.1 Génération du modèle de substitution

In [ ]:
configure_logger()

In [ ]:
default_discipline = H2TurboFan()
print(f"INPUT: {default_discipline.get_input_data_names()}\n")
print(f"OUPUT: {default_discipline.get_output_data_names()}\n")
print(f"DEFAULT VALUES: {default_discipline.default_inputs}\n")

The design parameters are :

-  the engine maximum thrust (100 kN ≤ thrust ≤ 150 kN, default: 125 kN),
- the engine bypass ratio (BPR) (5 ≤ BPR ≤ 12, default: 8.5),
- the wing area (120 m² ≤ area ≤ 200 m², default: 160 m²),
- the wing aspect ratio (7 ≤ ar ≤ 12, default: 9.5).


In [ ]:
default_discipline.execute()
aircraft_data = get_aircraft_data(default_discipline)
print(aircraft_data)
draw_aircraft(default_discipline, "The default A/C")

In [ ]:
design_space = create_design_space()
design_space.add_variable("thrust", l_b=1e5, u_b=1.5e5, value=1.25e5)
design_space.add_variable("bpr", l_b=5.0, u_b=12.0, value=8.5)
design_space.add_variable("area", l_b=120.0, u_b=200.0, value=160.0)
design_space.add_variable("aspect_ratio", l_b=7.0, u_b=12.0, value=9.5)

In [ ]:
dataset = sample_discipline(
    default_discipline,
    design_space,
    ["tofl", "vapp", "vz_mcl", "vz_mcr", "oei_path", "ttc", "far", "mtow"],
    "OT_OPT_LHS",
    30,
)

In [ ]:
avail_regressors = [
    "GaussianProcessRegressor",
    "GradientBoostingRegressor",
    "LinearRegressor",
    "MLPRegressor",
    "MOERegressor",
    "OTGaussianProcessRegressor",
    "PCERegressor",
    "PolynomialRegressor",
    "RBFRegressor",
    "RandomForestRegressor",
    "RegressorChain",
    "SVMRegressor",
    "TPSRegressor",
]

In [ ]:
surrogate_discipline = create_surrogate("RBFRegressor", dataset)
surrogate_discipline.execute()
surrogate_discipline.cache.last_entry

In [ ]:
# R2 measure
r2 = R2Measure(surrogate_discipline.regression_model, True)
r2.compute_learning_measure()
r2.compute_cross_validation_measure()

In [ ]:
ticks_labels = [
    "mtow",
    "tofl",
    "vapp",
    "vz_mcl",
    "vz_mcr",
    "oei_path",
    "ttc",
    "far",
]
plt.figure(figsize=(6, 4))
r2_data = r2.compute_cross_validation_measure()
plt.scatter(list(range(len(ticks_labels))), r2_data, color="blue")
plt.xticks(list(range(len(ticks_labels))), ticks_labels, rotation=45)
plt.ylabel("R2")
plt.ylim(0, 1)
plt.show()

In [ ]:
#  Root mean squared error
rmse = RMSEMeasure(surrogate_discipline.regression_model, True)
rmse.compute_learning_measure()
rmse.compute_cross_validation_measure()

## 1.2 Comparaison des modèles optimisés

### 1.2.1 Optimisation du modèle de référence

In [ ]:
scenario = create_scenario(
    [default_discipline], "DisciplinaryOpt", "mtow", design_space
)
scenario.add_constraint("tofl", constraint_type="ineq", positive=False, value=2200.0)
scenario.add_constraint(
    "vapp", constraint_type="ineq", positive=False, value=unit.mps_kt(137.0)
)
scenario.add_constraint(
    "vz_mcl", constraint_type="ineq", positive=True, value=unit.mps_ftpmin(300.0)
)
scenario.add_constraint(
    "vz_mcr", constraint_type="ineq", positive=True, value=unit.mps_ftpmin(0.0)
)
scenario.add_constraint(
    "oei_path", constraint_type="ineq", positive=True, value=1.1 * 1e-2
)
scenario.add_constraint(
    "ttc", constraint_type="ineq", positive=False, value=25.0 * 60.0
)
scenario.add_constraint("far", constraint_type="ineq", positive=False, value=13.4)

In [ ]:
scenario.execute({"algo": "NLOPT_COBYLA", "max_iter": 100})

In [ ]:
# Post-processing options
views = [
    "BasicHistory",
    "ConstraintsHistory",
    "Correlations",
    "GradientSensitivity",
    "ObjConstrHist",
    "OptHistoryView",
    "ParallelCoordinates",
    "ParetoFront",
    "QuadApprox",
    "RadarChart",
    "Robustness",
    "SOM",
    "ScatterPlotMatrix",
    "TopologyView",
    "VariableInfluence",
]
for view in views:
    try:
        scenario.post_process(view, save=False, show=True)
    except Exception:
        print(view)

### 1.2.2 Optimisation du modèle de substitution

In [ ]:
surrogate_scenario = create_scenario(
    [surrogate_discipline], "DisciplinaryOpt", "mtow", design_space
)

surrogate_scenario.add_constraint(
    "tofl", constraint_type="ineq", positive=False, value=2.2e3
)
surrogate_scenario.add_constraint(
    "vapp", constraint_type="ineq", positive=False, value=unit.mps_kt(137)
)
surrogate_scenario.add_constraint(
    "vz_mcl", constraint_type="ineq", positive=True, value=unit.mps_ftpmin(300)
)
surrogate_scenario.add_constraint(
    "vz_mcr", constraint_type="ineq", positive=True, value=0
)
surrogate_scenario.add_constraint(
    "oei_path", constraint_type="ineq", positive=True, value=1.1e-2
)
surrogate_scenario.add_constraint(
    "ttc", constraint_type="ineq", positive=False, value=unit.s_min(25)
)
surrogate_scenario.add_constraint(
    "far", constraint_type="ineq", positive=False, value=13.4
)

In [ ]:
surrogate_scenario.execute({"algo": "NLOPT_COBYLA", "max_iter": 10000})

In [ ]:
for view in views:
    try:
        surrogate_scenario.post_process(view, save=False, show=True)
    except Exception:
        print(view)

# 2. Analyse de sensibilité

## 2.1 Analyse des indices de Sobol

In [ ]:
class MyUncertainSpace(ParameterSpace):
    def __init__(self):
        super().__init__()
        self.add_random_variable(
            "tgi", "OTTriangularDistribution", minimum=0.25, mode=0.3, maximum=0.305
        )
        self.add_random_variable(
            "tvi", "OTTriangularDistribution", minimum=0.8, mode=0.845, maximum=0.85
        )
        self.add_random_variable(
            "drag", "OTTriangularDistribution", minimum=0.99, mode=1, maximum=1.03
        )
        self.add_random_variable(
            "sfc", "OTTriangularDistribution", minimum=0.99, mode=1, maximum=1.03
        )
        self.add_random_variable(
            "mass", "OTTriangularDistribution", minimum=0.99, mode=1, maximum=1.03
        )


uncertain_space = MyUncertainSpace()
print(uncertain_space)

### 2.1.1 Pour le modèle de référence

In [ ]:
sobol = SobolAnalysis([default_discipline], uncertain_space, 100)
sobol.compute_indices()

In [ ]:
pprint.pprint(sobol.first_order_indices)
pprint.pprint(sobol.total_order_indices)

In [ ]:
sobol.plot("far", save=False, show=True)

### 2.1.2 Pour le modèle de substitution

In [ ]:
uncertain_dataset = sample_discipline(
    default_discipline,
    uncertain_space,
    ["tofl", "vapp", "vz_mcl", "vz_mcr", "oei_path", "ttc", "far", "mtow"],
    "OT_OPT_LHS",
    30,
)

In [ ]:
names = ["mtow", "tgi", "tvi", "sfc", "mass", "drag"]
colors = ["blue", "red", "green", "orange", "purple", "brown"]
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
for i, (ax, name) in enumerate(zip(axes.flatten(), names)):
    ax.hist(uncertain_dataset.get_view(variable_names=name), bins=20, color=colors[i])
    ax.set_title(name)
plt.suptitle("Vérification des distributions des variables aléatoires")
plt.show()

In [ ]:
uncertain_surrogate_discipline = create_surrogate("RBFRegressor", uncertain_dataset)
uncertain_surrogate_discipline.execute()

In [ ]:
sobol_surrogate = SobolAnalysis(
    [uncertain_surrogate_discipline], uncertain_space, n_samples=10000
)
sobol_surrogate.compute_indices()

In [ ]:
sobol_surrogate.plot("mtow", save=False, show=True)

##  2.2 Analyse de Morris

In [ ]:
morris_analysis = MorrisAnalysis(
    [uncertain_surrogate_discipline], uncertain_space, n_samples=1000
)
morris_analysis.compute_indices()

In [ ]:
morris_analysis.plot("mtow", save=False, show=True, lower_mu=0, lower_sigma=0)

# 3. Gestion des incertitudes et optimisation

In [ ]:
full_design_space = create_design_space()
full_design_space.add_variable(
    "thrust", l_b=100.0 * 1e3, u_b=150.0 * 1e3, value=125 * 1e3
)
full_design_space.add_variable("bpr", l_b=5.0, u_b=12.0, value=8.5)
full_design_space.add_variable("area", l_b=120, u_b=200, value=160)
full_design_space.add_variable("aspect_ratio", l_b=7.0, u_b=12.0, value=9.5)
full_design_space.add_variable("tgi", l_b=0.25, u_b=0.305)
full_design_space.add_variable("tvi", l_b=0.8, u_b=0.85)
full_design_space.add_variable("drag", l_b=0.99, u_b=1.03)
full_design_space.add_variable("sfc", l_b=0.99, u_b=1.03)
full_design_space.add_variable("mass", l_b=0.99, u_b=1.03)

In [ ]:
sm = sample_discipline(
    default_discipline,
    full_design_space,
    ["tofl", "vapp", "vz_mcl", "vz_mcr", "oei_path", "ttc", "far", "mtow"],
    "OT_OPT_LHS",
    30,
)  # do it for 100

In [ ]:
umdo_surrogate_discipline = create_surrogate("RBFRegressor", sm)
umdo_surrogate_discipline.execute()
umdo_surrogate_discipline.cache.last_entry

# R2 measure
r2 = R2Measure(umdo_surrogate_discipline.regression_model, True)
r2.compute_learning_measure()
r2.compute_cross_validation_measure()

In [ ]:
umdo_surrogate_discipline.execute()

In [ ]:
design_space = create_design_space()
design_space.add_variable("thrust", l_b=1e5, u_b=1.5e5, value=1.25e5)
design_space.add_variable("bpr", l_b=5, u_b=12, value=8.5)
design_space.add_variable("area", l_b=120, u_b=200, value=160)
design_space.add_variable("aspect_ratio", l_b=7, u_b=12, value=9.5)

In [ ]:
uncertain_space = MyUncertainSpace()

In [ ]:
umdo_scenario = UMDOScenario(
    [umdo_surrogate_discipline],
    "DisciplinaryOpt",
    "mtow",
    design_space,
    uncertain_space,
    "Mean",
    statistic_estimation="Sampling",
    statistic_estimation_parameters={"n_samples": 100},
)

umdo_scenario.add_constraint(
    "tofl", "Margin", factor=3.0, constraint_type="ineq", positive=False, value=2200.0
)
umdo_scenario.add_constraint(
    "vapp",
    "Margin",
    factor=3.0,
    constraint_type="ineq",
    positive=False,
    value=unit.mps_kt(137.0),
)
umdo_scenario.add_constraint(
    "vz_mcl",
    "Margin",
    factor=3.0,
    constraint_type="ineq",
    positive=True,
    value=unit.mps_ftpmin(300.0),
)
umdo_scenario.add_constraint(
    "vz_mcr",
    "Margin",
    factor=3.0,
    constraint_type="ineq",
    positive=True,
    value=unit.mps_ftpmin(0.0),
)
umdo_scenario.add_constraint(
    "oei_path",
    "Margin",
    factor=3.0,
    constraint_type="ineq",
    positive=True,
    value=1.1e-2,
)
umdo_scenario.add_constraint(
    "ttc",
    "Margin",
    factor=3.0,
    constraint_type="ineq",
    positive=False,
    value=25.0 * 60.0,
)
umdo_scenario.add_constraint(
    "far",
    "Margin",
    factor=3.0,
    constraint_type="ineq",
    positive=False,
    value=13.4,
)

In [ ]:
umdo_scenario.set_differentiation_method("finite_differences")
umdo_scenario.execute({"algo": "NLOPT_SLSQP", "max_iter": 100})

In [ ]:
umdo_scenario.post_process("OptHistoryView", save=True, show=True)